In [2]:
import sys
import os

# Add the path to the aqi package
aqi_path = os.path.abspath('../../../airflow')
sys.path.append(aqi_path)

from plugins.hooks.aqi import fetch_data
from dags.aqi.etl import validate_data, parse_data, clean_data, push_to_db

In [3]:
aqi_data = fetch_data("6814")
print(aqi_data['status'])
print(len(aqi_data['data']))

<Response [200]>
{'status': 'ok', 'data': {'aqi': 95, 'idx': 6814, 'attributions': [{'url': 'http://aqmthai.com/', 'name': 'Division of Air Quality Data, Air Quality and Noise Management Bureau, Pollution Control Department.', 'logo': 'Thailand-PCD.png'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [13.5502631, 100.2647676], 'name': 'Provincial Administrative Organization, Samut Sakhon, Thailand (องค์การบริหารส่วนจังหวัดสมุทรสาคร)', 'url': 'https://aqicn.org/city/thailand/samut-sakhon/provincial-administrative-organization', 'location': ''}, 'dominentpol': 'pm25', 'iaqi': {'dew': {'v': -6.8}, 'h': {'v': 10}, 'no2': {'v': 7.5}, 'o3': {'v': 2.8}, 'p': {'v': 1015.6}, 'pm10': {'v': 33}, 'pm25': {'v': 95}, 'r': {'v': 99.8}, 'so2': {'v': 2.2}, 't': {'v': 27.6}, 'w': {'v': 2}}, 'time': {'s': '2025-02-24 20:00:00', 'tz': '+07:00', 'v': 1740427200, 'iso': '2025-02-24T20:00:00+07:00'}, 'forecast': {'daily': {'o3': [{'avg': 13, 'day': '2025-02-22', '

In [4]:
validate_data(aqi_data)

In [5]:
print(aqi_data["data"].get("idx"))

parse_aqi_data = parse_data(aqi_data)
print(parse_aqi_data)

6814
    idx                                            address  aqi  dew   h   o3  \
0  6814  Provincial Administrative Organization, Samut ...   95 -6.8  10  2.8   

        p  pm10  pm25     r     t  w                 timestamp  \
0  1015.6    33    95  99.8  27.6  2 2025-02-24 20:00:00+07:00   

                                            forecast  
0  {"o3": [{"avg": 13, "day": "2025-02-22", "max"...  


In [6]:
clean_aqi_data = clean_data(parse_aqi_data)
print(clean_aqi_data)

    idx                                            address  aqi  dew   h   o3  \
0  6814  Provincial Administrative Organization, Samut ...   95 -6.8  10  2.8   

        p  pm10  pm25     r     t  w                 timestamp  \
0  1015.6    33    95  99.8  27.6  2 2025-02-24 20:00:00+07:00   

                                            forecast  
0  {"o3": [{"avg": 13, "day": "2025-02-22", "max"...  


In [11]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': 'air_data',
    'user': 'admin',
    'password': 'adminpassword',
    'host': 'localhost',
    'port': '54329'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Push the cleaned AQI data to the database
push_data = push_to_db(conn, clean_aqi_data)

# Close the connection
conn.close()